# Organisateru d'image


### import

In [44]:
import os
import shutil
from datetime import datetime
from PIL import Image, ExifTags

### Env variable

In [62]:
imgExtensions = [".png", ".gif", ".jpg", ".jpeg", ".jfif", ".pjpeg", ".pjp"]
inputPath = "./New folder/Camera"
outputPath = "./Output Folder/"

months = ['janvier', 'fevrier', 'mars', 'avril', 'mai', 'juin', 'juillet', 'aout', 'septembre', 'octobre', 'novembre', 'decembre']

### Functions

In [75]:
def getFolderContent(input_path):
    output= {'files': [], 'directories': []}
    for f in os.listdir(input_path):
        if os.path.isfile(os.path.join(input_path, f)):
           output['files'].append(f)
        else: 
           output['directories'].append(f)
    
    return {'files': output['files'], 'directories': output['directories']}

def getFiles(input_path):
    content = getFolderContent(input_path)
    output = [f"{input_path}/{file}" for file in content["files"]]
    # output = f"{input_path}/{content['files']}"
    for dir in content['directories']:
        output = output + getFiles(os.path.join(input_path, dir))
    return output

def isImage(input_path):
    if(os.path.isdir(input_path) or os.path.isfile(input_path)):
        return os.path.splitext(input_path)[1] in imgExtensions

def getMetaData(input_path):
    if(isImage(input_path)):
        img = Image.open(os.path.abspath(input_path))
        return { ExifTags.TAGS[k]: v for k, v in img._getexif().items() if k in ExifTags.TAGS } if img._getexif() else {}
    else: return {}
    
def createOutputDatePath(output_path, timestamp):
    yearPath = os.path.join(output_path, timestamp[:4])
    monthPath = os.path.join(yearPath, f'{timestamp[5:7]}-{months[int(timestamp[5:7])-1]}')
    if(not os.path.exists(output_path)): os.mkdir(output_path)
    if(not os.path.exists(yearPath)): os.mkdir(yearPath)
    if(not os.path.exists(monthPath)): os.mkdir(monthPath)
    return monthPath





In [77]:
for file in getFiles("./Sample"):
    meta = getMetaData(file)
    if("DateTime" in meta):
        shutil.copyfile(file, os.path.join("Sorted", createOutputDatePath(outputPath, meta["DateTime"]), os.path.split(file)[1]))
    else:
        if(not os.path.exists(os.path.join("no meta data", outputPath))): os.mkdir(os.path.join("no meta data", outputPath))
        shutil.copyfile(file, os.path.join("no meta data", os.path.split(file)[1]))

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'no meta data\\./Output Folder/'